# Branch and Bound Algorithm for Solving Mixed Integer Programs (MIPs) or Binary Integer Programs (BIPs)

- The idea of the Branch & Bound algorithm is to indirectly arrive at an integer solution by running multiple linear programs that do not have constraints on the integer variables.  Multiple constraints are successively added that in some cases force solutions to have integer decision variables.  
- Integer solutions thus obtained may or may not be an optimal solution because the constraints break the preceding model into two parts: the other subpart might have a better solution.  These integer solutions are, therefore, a lower bound (__bound__) on the best possible solution.
- Non-integer solutions for a subproblem are possible upper bounds on the best possible objective function value if the solution is less than previous upper bounds (__bound__).
- Breaking an existing problem into two with two mutually exclusive but exhaustive constraints is called __branching__.
- Termination of a branch: __pruning__
  - A branch does not need to investigated further if its solution, either integer or not, when its solution is less than the current lower bound
  - A branch does not need to be investigated further if the problem is infeasible: no other subproblems on that branch will be feasible.
- Termination of the algorithm
  - The optimal solution has been found when the current integer solution has an objective function (lower bound) greater than the solutions on all other branches (which, of course, have been pruned).

## Example Problem

Your company makes two pieces of furniture: a sofa and a chair. Each sofa sold brings in \$20 (in hundreds) and each chair brings in \$10 (in hundreds) of revenue. Production of these two products consumes two different resource, which we will generically call "resource 1" and "resource 2". You have 120 units of resource 1 and 160 units of resource two available for the next production cycle. Each sofa requires 4 units of resource 1 and 7 units of resource 2. Each chair requires 3 units of resource 1 and 2 units of resource 2. Additionally, you want to produce at most 32 chairs during the next production cycle. 

We can formulate this problem as:

| | | |
| --- | --- | --- |
| Let | | |
| $x_{s}$ | = | number of sofas to make next production cycle |
| $x_{c}$ | = | number of chairs to make next production cycle |

| | | | | | | |
| --- | --- | --- | --- | --- | --- | --- |
| $\max$ | $20x_{s}$ | $+$ | $10x_{c}$ | | | |
| s.t. | $4x_{s}$ | $+$ | $3x_{c}$ | $\le$ | $120$ | {resource 1} |
| | $7x_{s}$ | $+$ | $2x_{c}$ | $\le$ | $160$ | {resource 2} |
| | | | $x_{c}$ | $\le$ | $32$ | {maximum number of chairs to make} |
| | $x_{s}$ | | | $\ge$ | $0$ | {non-negativity} |
| | | | $x_{c}$ | $\ge$ | $0$ | {non-negativity} |

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Define data
constraint_matrix = [[4,3], [7,2], [0,1]]
rhs = [120, 160, 32]
obj_fn_coeff = [20, 10]
products = ['sofa', 'chair']

In [ ]:
# create the Gurobi model
m = gp.Model('show_b_and_b')

# specify model sense
m.ModelSense = GRB.MAXIMIZE

# Create decision variables
x = [m.addVar(vtype=GRB.CONTINUOUS, name=f'x_{products[i]}') for i in range(len(products))]

# update
m.update()

# Create objective function
m.setObjective(gp.quicksum(obj_fn_coeff[i]*x[i] for i in range(len(obj_fn_coeff))))

# Create the constraints
for i in range(len(rhs)):
    m.addConstr((gp.quicksum(constraint_matrix[i][j]*x[j] for j in range(len(constraint_matrix[i]))) <= rhs[i]),
                name=f'constr_{i}')

m.update()
m.display()

In [ ]:
# solve it
m.optimize()

# print results
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value: {var.X}, (LB,UB): ({var.lb}, {var.ub})')

## Build a Function to Create the Base LP

We will need to run "many" optimizations based on the base LP relaxation. Therefore, it makes sense to write a function that can build and return this model. The function should return a Gurobi model (`m`) and the decision variables (`x`) that are needed to add constraints in the branch and bound algorithm.

In [ ]:
def create_base_lp(A, b, c, name='fun'):
    '''
    This function creates the base LP model. It has the following 

    Parameters:
    ============
    A : the constraint coefficient matrix
    b : the right-hand-side coefficients for constraints
    c : coefficient in the objective function
    name : default is 'fun'

    returns:
    ==========
    m : the Gurobi model object
    x : the variables
    '''
    import gurobipy as gp
    from gurobipy import GRB

    # Create the Gurobi model
    m = gp.Model(name)

    # specify model sense
    m.ModelSense = GRB.MAXIMIZE

    # Create variables
    x = m.addVars(len(A[0]), vtype=GRB.CONTINUOUS, name='x')

    # Create objective function
    m.setObjective(gp.quicksum(c[i]*x[i] for i in range(len(c))))

    # Create constraints
    for i in range(len(b)):
        m.addConstr((gp.quicksum(A[i][j]*x[j] for j in range(len(A[i]))) <= b[i]),
                   name=f'constr_{i}')

    # update the model
    m.update()

    return m, x

In [ ]:
create_base_lp(constraint_matrix, rhs, obj_fn_coeff, "hello")

## Branch on $x_{s}$

Want to try two things: $\leq 18$ and $\geq 19$

### Add Constraint $x_{s} \leq 18$

In [ ]:
m, x = create_base_lp(constraint_matrix, rhs, obj_fn_coeff)

m.addConstr(x[0] <= 18)
m.update()
m.optimize()

# print results
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value: {var.X}, (LB,UB): ({var.lb}, {var.ub})')

## Branch on $x_{s}$

Want to try two things: $\leq 18$ and $\geq 19$

### Add Constraint $x_{s} \geq 19$

In [ ]:
m, x = create_base_lp(constraint_matrix, rhs, obj_fn_coeff)

m.addConstr(x[0] >= 19)
m.update()
m.optimize()

# print results
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value: {var.X}, (LB,UB): ({var.lb}, {var.ub})')

## Verifying by Solving the IP

Let's verify that $x_s=18$ and $x_c=16$ with an objective function value of \$520 (hundreds) is the optimal integer solution by creating integer variables in the model and solving it that way.

In [ ]:
# create the Gurobi model
m_ip = gp.Model('verify_IP_soln')

# specify model sense
m_ip.ModelSense = GRB.MAXIMIZE

# Create decision variables
x = [m_ip.addVar(vtype=GRB.INTEGER, name=f'x_{products[i]}') for i in range(len(products))]

# update
m_ip.update()

# Create objective function
m_ip.setObjective(gp.quicksum(obj_fn_coeff[i]*x[i] for i in range(len(obj_fn_coeff))))

# Create the constraints
for i in range(len(rhs)):
    m_ip.addConstr((gp.quicksum(constraint_matrix[i][j]*x[j] for j in range(len(constraint_matrix[i]))) <= rhs[i]),
                name=f'constr_{i}')

m_ip.update()
m_ip.display()

In [ ]:
# solve it
m_ip.optimize()

# print results
for var in m_ip.getVars():
    print(f'Variable: {var.varName}, Optimal Value: {var.X}, (LB,UB): ({var.lb}, {var.ub})')

**&copy; 2024 - Present: Matthew D. Dean, Ph.D.   
Clinical Professor of Business Analytics at William \& Mary.**